# MLP for spuristo

In [95]:
import matplotlib.pyplot as plt
import numpy as np
from random import randint

## Classifier setup

This is a notebook used for testing and finding the optimal hyperparameters for our regression model

In [96]:
# TODO: fetch real data and divide into train, dev and test sets

data = [2, 8, 6, 4, 9, 3, 12]
data = [(randint(1, 12), randint(1, 31), randint(0, 6), randint(0, 23), i) for i in data]  # (month, day, weekday, hour, devices)
labels = [0, 3, 3, 2, 4, 1, 9]

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

## Fetch data

In [202]:
class MLP:
    def __init__(self, data, labels, hidden_layers=3, eta=0.01):
        self.original_data = data
        self.data = data
        self.labels = labels
        self.hidden_layers = hidden_layers
        self.eta = eta

        self.dim_in = self.data.shape[1]
        self.dim_out = self.labels.shape[0]

    def fit(self, epochs=100):
        # Initialize weights
        self.weights1 = np.random.rand(self.dim_in + 1, self.hidden_layers)
        self.weights2 = np.random.rand(self.hidden_layers, self.dim_out)
        #self.weights1_2 = (np.random.rand(self.dim_in + 1, self.hidden_layers) * 2 - 1)/np.sqrt(self.dim_in)
        #self.weights2 = (np.random.rand(self.hidden_layers+1, self.dim_out) * 2 - 1)/np.sqrt(self.hidden_layers)

        # scale data
        self.data = self._scale(self.data)

        # add bias to data
        self.data = self._add_bias(self.data)

        for _ in range(epochs):
            hidden_activations, output_activations = self._forward(self.data)
            self._backward(hidden_activations, output_activations)
    
    def _predict(self, x):
        _, output_activations = self._forward(x)
        return output_activations

    def predict(self, x):
        x = self._add_bias(self._scale(x))  # scale and add bias
        _, output_activations = self._forward(x)
        return output_activations

    def accuracy(self, x_test, y_test):
        x_test = self._add_bias(self._scale(x_test))  # scale and add bias
        output = [np.argmax(prediction) for prediction in self._predict(x_test)]
        return np.mean(output == y_test)

    def _add_bias(self, x):
        if len(x.shape) == 1:
            return np.concatenate([np.array([-1]), x])

        bias = np.negative(np.ones((self.data.shape[0], 1)))
        #return np.concatenate((x, bias))
        return np.append(x, bias, axis=1)

    def _forward(self, x):
        #hidden_activations = self._add_bias(self._sigmoid(x @ self.weights1))
        hidden_activations = self._sigmoid(x @ self.weights1)
        output_activations = self._sigmoid(hidden_activations @ self.weights2)
        return hidden_activations, output_activations

    def _backward(self, hidden_activations, output_activations):
        output = (output_activations - self.labels) * output_activations * (1 - output_activations)
        hidden = self._activation_derived(1 - output_activations) * output @ self.weights2.T
        #hidden = hidden[:, 1:]  # remove bias
        #hidden = hidden[:, :]  # remove bias

        self.weights2 -= self.eta * hidden_activations.T @ output
        self.weights1 -= self.eta * self.data.T @ hidden

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _scale(self, x):
        return (x - np.min(x)) / (np.max(x) - np.min(x))

    def _activation_derived(self, x):
        return x * (1 - x)


## Training

In [203]:
clf = MLP(data, labels)
clf.fit(epochs=1000)

## Results

In [212]:
new_data = np.array([10, 19, 3, 14, 20])
print(clf.predict(new_data))

#plt.scatter(clf.data, clf.labels)
#model = list(map(clf.predict, clf.data))
#plt.plot(clf.data, model)

#plt.show()

[0.1459464  0.97876115 0.97675236 0.96676205 0.98435305 0.86664799
 0.99165184]
